# SORRY, still work in progress, very messy and not finished

In [89]:
import sys
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import c3d
import importlib
from typing import List

# Import RGB_helpers
sys.path.append('/app/hpe-core/datasets/vicon_processing/v2')
import RGB_helpers

In [90]:
vicon_c3d_file: str = '/home/cappe/hpe/RGB_calibration/test6_vicon/test6.c3d'
RGB_folder: str = '/home/cappe/hpe/RGB_calibration/test6/rgb.mp4'
calib_path: str = '/home/cappe/hpe/RGB_calibration/color_camera.yaml' 
cap = cv2.VideoCapture(RGB_folder)
fps = cap.get(cv2.CAP_PROP_FPS)
period: float = 1.0 / fps  # vicon frequency: 100HZ
input_label_tag_file: str = '../scripts/config/labels_tags_vicon.yml'
output_label_yml_file: str = '/home/cappe/hpe/RGB_calibration/labels_RGB_test6.yml'

### Import RGB data and calibration

In [94]:
import os
import yaml
import numpy as np

def load_camera_calibration(yaml_path, camera="color"):
    """
    Load camera intrinsics (K, D) from a ROS camera_info-style YAML file.
    
    Args:
        yaml_path: path to the yaml file
        camera: camera name ('color' or 'depth') if both are stored
    
    Returns:
        K (3x3), D (Nx1), image_size (width, height)
    """
    with open(yaml_path, "r") as f:
        data = yaml.safe_load(f)

    if data.get("camera_name", "") != camera:
        print(f"⚠️ Warning: loaded camera_name={data.get('camera_name')} but expected {camera}")

    # Camera intrinsics
    K = np.array(data["camera_matrix"]["data"], dtype=np.float64).reshape(3, 3)

    # Distortion coefficients
    D = np.array(data["distortion_coefficients"]["data"], dtype=np.float64).reshape(-1, 1)

    width = data.get("image_width", None)
    height = data.get("image_height", None)

    return K, D, (width, height)


# Open video (reuse RGB_folder from previous cell)
cap = cv2.VideoCapture(RGB_folder)
if not cap.isOpened():
    raise RuntimeError(f"Cannot open video: {RGB_folder}")

# Read video metadata and set period
fps = cap.get(cv2.CAP_PROP_FPS) or 100.0
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)
period = 1.0 / float(fps) if fps > 1e-6 else 1.0 / 100.0

# Image sizes used elsewhere: cam_res = (H,W) for grayscale/event buffers
cam_res = (h, w)
cam_res_rgb = (h, w, 3)

print(f"Video: {frame_count} frames, {fps:.2f} FPS, size={w}x{h} -> period={period:.6f}s")

K, D, (w_calib, h_calib) = load_camera_calibration(calib_path, camera="color")

print("K:\n", K)
print("D:\n", D)
print("Image size:", (w_calib, h_calib))

print("Camera matrix:\n", K)
print("Distortion coeffs:", D)
print("cam_res:", cam_res)


cap = cv2.VideoCapture(RGB_folder)  # RGB_folder = path to .mp4 or .avi
if not cap.isOpened():
    raise RuntimeError(f"Cannot open video: {RGB_folder}")

fps = cap.get(cv2.CAP_PROP_FPS) or 30.0
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) or 0)
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) or 0)
period = 1.0 / fps

print(f"Video: {frame_count} frames, {fps:.2f} FPS, {w}x{h}")

Video: 2545 frames, 30.00 FPS, size=1280x720 -> period=0.033333s
K:
 [[920.57775879   0.         643.53894043]
 [  0.         918.72113037 353.19778442]
 [  0.           0.           1.        ]]
D:
 [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Image size: (1280, 720)
Camera matrix:
 [[920.57775879   0.         643.53894043]
 [  0.         918.72113037 353.19778442]
 [  0.           0.           1.        ]]
Distortion coeffs: [[0.]
 [0.]
 [0.]
 [0.]
 [0.]]
cam_res: (720, 1280)
Video: 2545 frames, 30.00 FPS, 1280x720


### Import C3D data

In [78]:
c3d_data = c3d.Reader(open(vicon_c3d_file, 'rb'))
points_3d: dict = {}
for i, points, analog in c3d_data.read_frames():
    points_3d[i] = points

marker_t = np.linspace(0.0, c3d_data.frame_count / c3d_data.point_rate, 
                            c3d_data.frame_count, endpoint=False)

c3d_data.point_labels

/home/cappe/hpe/hpe-core/datasets/vicon_processing/v2/.venv/lib/python3.10/site-packages/c3d/c3d.py:1219: UserWarning: No analog data found in file.
  warnings.warn('No analog data found in file.')


array(['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
       'stereoatis:cam_left           ', 'box1:top_1_origin             ',
       'box1:top_2                    ', 'box1:top_3                    ',
       'box1:top_4                    ', 'box1:bottom_1                 ',
       'box1:bottom_2                 ', 'box1:bottom_3                 ',
       'box1:bottom_4                 ', '*11                           '],
      dtype=object)

### Initial delay definition

In [79]:
delay = 0.365     # adjust accordingly

### Label sequence

In [6]:
import importlib
import os
import cv2
import numpy as np

importlib.reload(RGB_helpers)
from RGB_helpers import RGBLabeler

# Path setup
labels_path = os.path.join(os.path.dirname(vicon_c3d_file), output_label_yml_file)

# Choose labeling period (seconds per labeling step, e.g. 0.05s = 20Hz)
period = 0.05  

# Initialize labeler with video file
labeler = RGBLabeler(video_path=RGB_folder, subject="camera")

# Run interactive labeling
labeled_dict = labeler.label_video(RGB_folder, period)

# Save labeled points to YAML
labeler.save_labeled_points(labels_path)

print("Saved labeled points at:", labels_path)

cv2.destroyAllWindows()

current labels: {'camera:box1:bottom_1': {'x': 670, 'y': 362}}
Removed last labeled marker.
current labels: {'camera:box1:bottom_1': {'x': 667, 'y': 360}}
current labels: {'camera:box1:bottom_1': {'x': 667, 'y': 360}, 'camera:box1:top_1_origin': {'x': 606, 'y': 28}}
current labels: {'camera:box1:bottom_1': {'x': 667, 'y': 360}, 'camera:box1:top_1_origin': {'x': 606, 'y': 28}, 'camera:box1:top_2': {'x': 294, 'y': 184}}
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
sp

### Load labels and calculate transformation matrix from system to camera using scipy optimizer

In [80]:
from scipy.optimize import least_squares

def reprojection_error(params, Ps, pc, K, dist):
    """
    params: 6 dimensions (rotation_vector [3], translation [3]) Optimization target
    Ps: (N, 3) 3D points (system coordinates)
    pc: (N, 2) 2D points (image coordinates)
    K: (3, 3) camera matrix
    dist: (5,) distortion parameters (OpenCV format)
    """
    rvec = params[:3]
    tvec = params[3:6]

    # Project 3D system points to 2D image points
    projected_points, _ = cv2.projectPoints(Ps, rvec, tvec, K, dist)
    projected_points = projected_points.reshape(-1, 2)

    return (projected_points - pc).ravel()  # Flatten and return


def estimate_Tstoc(Ps, pc, K, dist, init_params=None):
    """
    Ps: (N, 3) 3D points (system coordinates)
    pc: (N, 2) 2D points (image coordinates)
    K: (3, 3) camera matrix
    dist: (5,) distortion parameters
    return: (4, 4) system→camera coordinate transformation matrix Tstoc
    """

    # Initial values: zero rotation, zero translation
    if init_params is None:
        # If no initial parameters are provided, set them to zero
        init_params = np.zeros(6)

    # Minimize (choose LM method, etc.)
    res = least_squares(
        reprojection_error,
        init_params,
        args=(Ps, pc, K, dist),
        method='lm'  # Levenberg-Marquardt
    )

    rvec_opt = res.x[:3]
    tvec_opt = res.x[3:6]
    R_opt, _ = cv2.Rodrigues(rvec_opt)

    # Construct homogeneous transformation matrix
    T = np.eye(4)
    T[:3, :3] = R_opt
    T[:3, 3] = tvec_opt

    return T

In [87]:
importlib.reload(RGB_helpers)
from RGB_helpers import ViconHelperRGB

# Define vicon helper
print("Delay: ", delay) # Please adjust delay manually
vicon_helper = ViconHelperRGB(marker_t, points_3d, delay, c3d_data.frame_count, c3d_data.point_rate, c3d_data.point_labels, True, True) # if needed create the function to call instead of None

# Load labeled points
labels_path = os.path.join(os.path.dirname(vicon_c3d_file), output_label_yml_file)
labeled_points = RGB_helpers.read_points_labels(labels_path) # read labeled points from the yaml file

# extract the frames_id from the labeled points
frames_id = vicon_helper.get_frame_time(labeled_points['times'])
vicon_points = vicon_helper.get_vicon_points_interpolated(labeled_points)
# print("First few label dicts from YAML:")
# for i, d in enumerate(labeled_points['points']):
#     print(f"Frame {i}: {d}")

# Create lists of transformation matrix from world to system
system_points = []
image_points = []
Ts_system_to_camera: List[np.ndarray] = []
Ts_world_to_system = vicon_helper.compute_camera_marker_transforms()

# Collect correspondences from 3D system points to 2D image points
for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            w_p = vicon_frame[l]
            w_ph = np.append(w_p, 1.0)  # homogeneous coordinates
            i_p = [
                dvs_frame[l]['x'],
                dvs_frame[l]['y'] #,
                # 1.0
            ]
            frame_id = vicon_points['frame_ids'][idx]
            p_sys = Ts_world_to_system[frame_id] @ w_ph  # convert to system coordinates
            system_points.append(p_sys[:3])
            image_points.append(i_p)
        except Exception as e:
            print("The stored image labels probably don't match with the vicon labels used.")
            print(e)
system_points = np.array(system_points, dtype=np.float64)
image_points = np.array(image_points, dtype=np.float64)
image_points = image_points[:, :2]



Delay:  0.365
out {'points': [{'stereoatis:cam_right': array([ 33.140896, 576.4291  , 613.6085  ], dtype=float32), 'stereoatis:cam_back': array([-39.106777, 607.68024 , 668.3448  ], dtype=float32), 'stereoatis:cam_left': array([-14.715498, 578.403   , 597.232   ], dtype=float32)}, {'stereoatis:cam_right': array([ 33.120537, 576.43243 , 613.6466  ], dtype=float32), 'stereoatis:cam_back': array([-39.108475, 607.6763  , 668.38116 ], dtype=float32), 'stereoatis:cam_left': array([-14.72231, 578.40076, 597.264  ], dtype=float32)}, {'stereoatis:cam_right': array([ 33.104424, 576.4735  , 613.68823 ], dtype=float32), 'stereoatis:cam_back': array([-39.132492, 607.705   , 668.393   ], dtype=float32), 'stereoatis:cam_left': array([-14.735522, 578.41644 , 597.2791  ], dtype=float32)}, {'stereoatis:cam_right': array([ 33.077858, 576.47015 , 613.6858  ], dtype=float32), 'stereoatis:cam_back': array([-39.14639, 607.6862 , 668.3925 ], dtype=float32), 'stereoatis:cam_left': array([-14.761955, 578.436   

### Static projection???

In [105]:
# Collect correspondences with better error handling and then do projection
world_points = []
image_points_clean = []

marker_names = ['box1:top_1_origin             ', 'box1:top_2                    ',
                'box1:top_3                    ', 'box1:top_4                    ',
                'box1:bottom_1                 ', 'box1:bottom_2                 ',
                'box1:bottom_3                 ', 'box1:bottom_4                 '] 

for idx, (dvs_frame, vicon_frame) in enumerate(zip(labeled_points['points'], vicon_points['points'])):
    labels = dvs_frame.keys()

    for l in labels:
        try:
            if l not in vicon_frame:
                print(f"Warning: Marker '{l}' not found in Vicon frame {idx}")
                continue
                
            w_p = vicon_frame[l]
            
            # Check if the point is valid (not NaN or None)
            if w_p is None or np.any(np.isnan(w_p)):
                print(f"Warning: Invalid 3D point for marker '{l}' in frame {idx}")
                continue
                
            i_p = [dvs_frame[l]['x'], dvs_frame[l]['y']]
            
            # Use world coordinates directly
            world_points.append(w_p)
            image_points_clean.append(i_p)
            
        except KeyError as e:
            print(f"KeyError: Marker '{l}' not found in Vicon data at frame {idx}")
        except Exception as e:
            print(f"Error processing marker '{l}': {e}")

world_points = np.array(world_points, dtype=np.float64)
image_points_clean = np.array(image_points_clean, dtype=np.float64)

print(f"Successfully collected {len(world_points)} valid 3D-2D correspondences")
print("World points shape:", world_points.shape)
print("Image points shape:", image_points_clean.shape)

# Solve PnP with world coordinates
success, rvec, tvec = cv2.solvePnP(world_points, image_points_clean, K, D)

if success:
    print("PnP solution successful!")
    
    # Convert to transformation matrix
    R_mat, _ = cv2.Rodrigues(rvec)
    T_world_to_cam = np.eye(4)
    T_world_to_cam[:3, :3] = R_mat
    T_world_to_cam[:3, 3] = tvec.flatten()
    
    print("World-to-camera transformation matrix:")
    print(T_world_to_cam)
    
    # Test projection of a few points
    if len(world_points) > 0:
        test_points = world_points[:min(5, len(world_points))]
        projected_test, _ = cv2.projectPoints(test_points, rvec, tvec, K, D)
        projected_test = projected_test.reshape(-1, 2)
        
        print("Test projections (first few points):")
        for i in range(len(projected_test)):
            print(f"3D: {test_points[i]} -> 2D projected: {projected_test[i]} (labeled: {image_points_clean[i]})")   

    print("Projecting markers onto event plane...")
    
    # TODO: adjust for RGB data
    projected_points_pnp = RGB_helpers.project_vicon_to_rgb_plane(
        marker_names, c3d_data, points_3d, marker_t, T_world_to_cam, 
        {i: np.eye(4) for i in range(len(marker_t))},  # Identity transforms since we're using world-to-camera directly
        K, RGB_folder, delay, D=D, visualize=True
    )
    
else:
    print("PnP solution failed!")
    print("Check that you have enough valid correspondences and that the points are not coplanar")

Successfully collected 92 valid 3D-2D correspondences
World points shape: (92, 3)
Image points shape: (92, 2)
PnP solution successful!
World-to-camera transformation matrix:
[[ 2.52084960e-01 -9.64584380e-01 -7.76540206e-02  9.39121455e+02]
 [-3.33502738e-01 -1.12670209e-02 -9.42681801e-01  8.96706707e+02]
 [ 9.08421211e-01  2.63533733e-01 -3.24531778e-01  2.18402405e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Test projections (first few points):
3D: [ 560.69891357 1058.85693359  635.91906738] -> 2D projected: [654.72575725 466.0406098 ] (labeled: [667. 360.])
3D: [ 560.07989502 1048.93530273  876.12011719] -> 2D projected: [644.16390441 189.96125796] (labeled: [606.  28.])
3D: [ 509.61108398 1406.04040527  881.8807373 ] -> 2D projected: [214.15247384 208.70383627] (labeled: [294. 184.])
3D: [ 560.74859619 1058.78479004  635.91003418] -> 2D projected: [654.82059376 466.02820332] (labeled: [667. 362.])
3D: [ 560.05072021 1048.93273926  876.18353271] -> 2D pr

In [96]:
#TODO: change this so that it is not hardcoded bit instead generated by user inputs

# Create initial guess for the optimization
from scipy.spatial.transform import Rotation

# init_T = np.array([[ 5.54118388e-01, -6.46688971e-01, 5.24162298e-01, -1.25350531e+02],
#                    [-3.65516271e-01, -7.54740636e-01, -5.44760888e-01, -8.06780263e+00],
#                    [ 7.47897430e-01, 1.10272191e-01, -6.54591006e-01, -9.41323412e+01],
#                    [ 0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

# init_T = np.array([[ 9.58177085e-01,  2.70140431e-01,  9.44500986e-02, -1.10010569e+03],
#                    [ 9.62649522e-02, -6.15063556e-01,  7.82578993e-01, -7.74416582e+02],
#                    [ 2.69499040e-01, -7.40757024e-01, -6.15344861e-01,  4.56237436e+02],
#                    [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

init_T = T_world_to_cam @ np.linalg.inv(Ts_world_to_system[0])

r_vec = Rotation.from_matrix(init_T[:3, :3]).as_rotvec()
t_vec = init_T[:3, 3]
r_vec, t_vec, init_T
init_param = np.concatenate((r_vec, t_vec))
init_param

array([ -2.37968262,   1.06367994,  -1.51089203, 331.11507165,
       332.09314475, 163.64120934])

In [97]:
T_w2s_opt = estimate_Tstoc(system_points, image_points, K, D, init_param)
T_w2s_opt

#TODO: save the transformation matrix and the delay in a file

array([[   0.55805074,   -0.63049953,    0.53949024,   18.58643793],
       [  -0.27490281,   -0.75389659,   -0.59671465, -143.42430942],
       [   0.78294816,    0.18468966,   -0.59403865,  -28.75077275],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

### Manual rotation adjustment

In [ ]:
marker_names = ['P1:LANK', 'P1:RANK', 'P1:LKNE', 'P1:RKNE', 'P1:RSHO', 'P1:LSHO', 'P1:LELB', 'P1:RELB', 'P1:LWRA', 'P1:RWRA', 'P1:CLAV', 'P1:STRN', 'P1:LFHD', 'P1:RFHD']
chosen_one = 'P1:CLAV'  # choose the marker to visualize

# TODO: adjust for RGB data

projected_points = RGB_helpers.project_and_manual_rotation(
    marker_names, c3d_data, points_3d, marker_t, np.eye(4), Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D, chosen_one=chosen_one
)

### Correct Delay by going though all frames

In [ ]:
importlib.reload(RGB_helpers)

#TODO: modify function

# marker_names = ['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
#        'stereoatis:cam_left           ', 'monitor:Origin_mon_top_right  ',
#        'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
#        'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
#        'monitor:base_2                ', 'monitor:base_3                ',
#        'monitor:base_4                ', 'chair:origin_right_top        ',
#        'chair:right_mid               ', 'chair:right_bottom            ',
#        'chair:left_top                ', 'chair:left_bottom             ',
#        'box2:top_right_Origin         ', 'box2:bottom_right             ',
#        'box2:bottom_left              ', 'box2:top_left                 ',
#        'box1:top_1_origin             ', 'box1:top_2                    ',
#        'box1:top_3                    ', 'box1:top_4                    ',
#        'box1:bottom_1                 ', 'box1:bottom_2                 ',
#        'box1:bottom_3                 ', 'box1:bottom_4                 ']

marker_names = ['P1:LANK', 'P1:RANK', 'P1:LKNE', 'P1:RKNE', 'P1:RSHO', 'P1:LSHO', 'P1:LELB', 'P1:RELB', 'P1:LWRA', 'P1:RWRA', 'P1:CLAV', 'P1:STRN', 'P1:LFHD', 'P1:RFHD']

# TODO: adjust for RGB data

projected_points = RGB_helpers.fix_delay(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
)

### Project points in dynamic scene

In [ ]:
importlib.reload(RGB_helpers)

# marker_names = ['stereoatis:cam_right          ', 'stereoatis:cam_back           ',
#        'stereoatis:cam_left           ', 'monitor:Origin_mon_top_right  ',
#        'monitor:mon_bottom_right      ', 'monitor:mon_bottom_left       ',
#        'monitor:mon_top_left          ', 'monitor:base_origin_1         ',
#        'monitor:base_2                ', 'monitor:base_3                ',
#        'monitor:base_4                ', 'chair:origin_right_top        ',
#        'chair:right_mid               ', 'chair:right_bottom            ',
#        'chair:left_top                ', 'chair:left_bottom             ',
#        'box2:top_right_Origin         ', 'box2:bottom_right             ',
#        'box2:bottom_left              ', 'box2:top_left                 ',
#        'box1:top_1_origin             ', 'box1:top_2                    ',
#        'box1:top_3                    ', 'box1:top_4                    ',
#        'box1:bottom_1                 ', 'box1:bottom_2                 ',
#        'box1:bottom_3                 ', 'box1:bottom_4                 ']

marker_names = ['P1:LANK', 'P1:RANK', 'P1:LKNE', 'P1:RKNE', 'P1:RSHO', 'P1:LSHO', 'P1:LELB', 'P1:RELB', 'P1:LWRA', 'P1:RWRA', 'P1:CLAV', 'P1:STRN', 'P1:LFHD', 'P1:RFHD']

# TODO: adjust for RGB data

projected_points = RGB_helpers.project_vicon_to_event_plane_dynamic(
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, K, cam_res, delay, e_ts, e_us, e_vs, period, visualize=True, D=D
)

### Adjust labels with on-screen projection of the points

In [ ]:
importlib.reload(RGB_helpers)
from RGB_helpers import DvsLabeler

# Define output path for corrections
corrections_path = os.path.join(os.path.dirname(vicon_c3d_file), 'corrections_labels.yml')

# Create corrector instance
corrector = DvsLabeler(img_shape=(720, 1280, 3))

# Batch correction across multiple frames
time_tags, event_indices = RGB_helpers.calc_indices(e_ts, period)

# Define marker names for correction
marker_names = ['P1:LANK', 'P1:RANK', 'P1:LWRA', 'P1:RWRA', 'P1:CLAV', 'P1:STRN', 'P1:LFHD', 'P1:RFHD', 'P1:RASI', 'P1:LASI', 'P1:LELB', 'P1:RELB', 'P1:LKNE', 'P1:RKNE', 'P1:RSHO', 'P1:LSHO']

# TODO: adjust for RGB data

corrector.correct_data(
    e_ts, e_us, e_vs, event_indices, time_tags, period, 
    marker_names, c3d_data, points_3d, marker_t, T_w2s_opt, Ts_world_to_system, 
    K, cam_res, delay, D
)

# Save corrected results automatically (using existing save_labeled_points method)
corrector.save_labeled_points(corrections_path)

print("Saved corrected points at:", corrections_path)

cv2.destroyAllWindows()

space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, skipping
space pressed, s